In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Conv2D, Dense
from tensorflow.keras import layers
import keras.backend as K
import sys
from tensorflow.keras.datasets import cifar10

np.set_printoptions(threshold=sys.maxsize)
gpus = tf.config.experimental.list_physical_devices('XLA_GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'XLA_GPU')

In [225]:
X_train1 = np.load('./images1.npy')
X_train2 = np.load('./images2.npy')
y_train = np.load('./array_data.npy')

In [226]:
print('X_train1 : ',X_train1.shape, 'X_train2 : ', X_train2.shape, 'y_train : ', y_train.shape)

X_train1 :  (5000, 224, 224, 3) X_train2 :  (5015, 224, 224, 3) y_train :  (10015,)


In [227]:
images = []

for i in range(len(X_train1)):
    images.append(X_train1[i])
    
for i in range(len(X_train2)):
    images.append(X_train2[i])
    
images = np.array(images)

images.shape

(10015, 224, 224, 3)

In [228]:
image_label = [[image, label] for image, label in zip(images, y_train)]

In [229]:
image_labels = np.array(image_label)

image_labels.shape

(10015, 2)

In [230]:
image_labels[0][1]

1

In [231]:
task0 = []
task1 = []
task2 = []
task3 = []
task4 = []
task5 = []
task6 = []

for i in range(len(image_labels)):
    if image_labels[i][1] == 0:
        task0.append(image_labels[i])
    elif image_labels[i][1] == 1:
        task1.append(image_labels[i])
    elif image_labels[i][1] == 2:
        task2.append(image_labels[i])
    elif image_labels[i][1] == 3:
        task3.append(image_labels[i])
    elif image_labels[i][1] == 4:
        task4.append(image_labels[i])
    elif image_labels[i][1] == 5:
        task5.append(image_labels[i])
    elif image_labels[i][1] == 6:
        task6.append(image_labels[i])
        
task0 = np.array(task0)
task1 = np.array(task1)
task2 = np.array(task2)
task3 = np.array(task3)
task4 = np.array(task4)
task5 = np.array(task5)
task6 = np.array(task6)

print(task0.shape,
task1.shape,
task2.shape,
task3.shape,
task4.shape,
task5.shape,
task6.shape)

(1113, 2) (6705, 2) (514, 2) (327, 2) (1099, 2) (115, 2) (142, 2)


In [232]:
def image_new(images):
    image = images
    image90 = np.rot90(images)
    image180 = np.rot90(images, 2)
    image270 = np.rot90(images, 3)
    
    return image, image90, image180, image270

In [253]:
image = []
image90 = []
image180 = []
image270 = []

for i in range(len(task6)):
    x, x1, x2, x3 = image_new(task6[i][0])
    image.append(x)
    image90.append(x1)
    image180.append(x2)
    image270.append(x3)
    
image = np.array(image)
image90 = np.array(image90)
image180 = np.array(image180)
image270 = np.array(image270)

print(image.shape, 
     image90.shape,
     image180.shape,
     image270.shape)

(142, 224, 224, 3) (142, 224, 224, 3) (142, 224, 224, 3) (142, 224, 224, 3)


In [254]:
yhats = [6] * (142 * 4)

In [255]:
yhats = np.array(yhats)

yhats.shape

(568,)

In [256]:
task2_array = []

for i in range(142):
    task2_array.append(image[i])
    task2_array.append(image90[i])
    task2_array.append(image180[i])
    task2_array.append(image270[i])
    
task2_array = np.array(task2_array)

task2_array.shape

(568, 224, 224, 3)

In [257]:
task2_preprocessing = [[image, label] for image, label in zip(task2_array, yhats)]

task2_preprocessing = np.array(task2_preprocessing)

task6 = task2_preprocessing

In [258]:
print(task0.shape,
task1.shape,
task2.shape,
task3.shape,
task4.shape,
task5.shape,
task6.shape)

(1113, 2) (6705, 2) (2056, 2) (1308, 2) (1099, 2) (460, 2) (568, 2)


# 모든 데이터 합치고, 섞으면 train set 완성

In [306]:
train_data = []

for i in range(len(task0)):
    train_data.append(task0[i])
    
for i in range(len(task1)):
    train_data.append(task1[i])
    
for i in range(len(task2)):
    train_data.append(task2[i])
    
for i in range(len(task3)):
    train_data.append(task3[i])
    
for i in range(len(task4)):
    train_data.append(task4[i])
    
for i in range(len(task5)):
    train_data.append(task5[i])
    
for i in range(len(task6)):
    train_data.append(task6[i])
    
train_data = np.array(train_data)
train_data.shape

(13309, 2)

In [307]:
s = np.arange(13309)
np.random.shuffle(s)

nee = train_data[s]

In [308]:
train = []
label = []

for i in range(len(nee)):
    train.append(nee[i][0])
    label.append(nee[i][1])
    
train = np.array(train)
label = np.array(label)

print('train_shape : ', train.shape, 'label_shape : ', label.shape)

train_shape :  (13309, 224, 224, 3) label_shape :  (13309,)


In [309]:
train = train / 255

X_train, y_train = train[:11000], label[:11000]
X_valid, y_valid = train[11000:], label[11000:]

In [310]:
print('X_train : ', X_train.shape, 
     'y_train : ', y_train.shape,
     'X_valid : ', X_valid.shape,
     'y_valid : ', y_valid.shape)

X_train :  (11000, 224, 224, 3) y_train :  (11000,) X_valid :  (2309, 224, 224, 3) y_valid :  (2309,)


In [311]:
from sklearn.utils import class_weight

weights = class_weight.compute_class_weight('balanced',
                                            np.unique(y_train),
                                            y_train)

weights = {i : weights[i] for i in range(7)}

weights

c:\programdata\anaconda3\envs\testpy36\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4 5 6], y=[3 1 1 1 1 1 1 1 1 1 1 1 3 6 0 2 1 6 4 1 1 3 3 1 3 1 1 5 2 1 1 6 2 4 1 1 1
 1 1 0 2 1 1 1 1 2 1 1 1 2 5 1 0 1 1 1 1 3 1 2 6 1 6 2 1 6 1 2 0 2 1 1 3 4
 3 4 2 5 1 1 1 1 1 1 3 1 1 0 4 2 2 6 1 2 0 1 2 0 1 1 3 4 2 2 1 1 1 2 2 2 1
 2 3 1 3 1 4 1 1 3 1 2 1 0 1 2 6 1 1 1 5 2 3 1 2 2 2 1 1 3 1 1 1 1 4 6 3 1
 6 1 0 0 3 0 4 1 1 1 5 1 2 6 4 3 0 1 1 1 1 3 1 5 4 4 6 1 1 1 4 3 2 1 3 4 2
 4 1 6 1 1 3 1 1 4 1 1 4 1 1 6 1 1 1 6 1 1 2 1 1 1 1 1 6 1 2 1 5 1 1 2 1 1
 6 1 4 1 1 1 1 1 6 1 1 3 2 4 1 1 2 1 1 3 5 4 1 0 3 2 0 2 2 3 0 6 1 1 2 3 6
 3 1 1 1 4 1 3 1 4 1 2 1 1 4 1 2 0 4 3 1 1 2 1 0 1 1 2 1 1 1 1 2 1 1 1 1 1
 1 1 1 1 1 5 6 1 2 2 1 2 2 2 6 1 3 1 4 1 1 0 1 1 2 2 3 0 0 0 1 1 1 4 1 1 0
 1 1 1 3 1 6 1 1 0 1 1 3 0 3 3 2 1 1 0 3 1 1 0 1 2 3 0 6 6 1 4 1 1 4 1 1 6
 1 1 4 2 2 1 1 1 5 1 1 2 2 1 3 0 1 1 1 4 2 1 2 4 3 3 2 2 0 2 2 4 0 1 1 1 3
 1 2 0 1 1 3 1 3 1 1 1 4 2 6 2 1 2 2 6 

{0: 1.7402309761113748,
 1: 0.282326369282891,
 2: 0.9337068160597572,
 3: 1.4590794535084228,
 4: 1.7080745341614907,
 5: 4.092261904761905,
 6: 3.3649434077699603}

In [332]:
model = tf.keras.models.load_model('./model.h5')

In [333]:
print(model.summary())
output = model.layers[-3].output
models = keras.models.Model(inputs = model.input, outputs = output)

Model: "SE-ResNet101"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [334]:
model.trainable = False

In [335]:
print(models.summary())

Model: "functional_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________

In [336]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
dense1 = tf.keras.layers.Dense(512, activation = 'relu')
drop1 = tf.keras.layers.Dropout(0.25)
dense2 = tf.keras.layers.Dense(256, activation = 'relu')
drop2 = tf.keras.layers.Dropout(0.25)
dense3 = tf.keras.layers.Dense(128, activation = 'relu')
drop3 = tf.keras.layers.Dropout(0.25)
prediction_layer = tf.keras.layers.Dense(7)

model = tf.keras.Sequential([
    models,
    global_average_layer,
    dense1,
    drop1,
    dense2,
    drop2,
    dense3,
    drop3,
    prediction_layer
])

In [337]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_27 (Functional)   (None, 7, 7, 2048)        79919488  
_________________________________________________________________
global_average_pooling2d_10  (None, 2048)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)             

In [338]:
len(model.trainable_variables)

8

In [339]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [340]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    batch_size = 8, 
                    class_weight=weights)

Epoch 1/100
1375/1375 [==============================] - 136s 99ms/step - loss: 1.7249 - accuracy: 0.3549 - val_loss: 1.7192 - val_accuracy: 0.3049
Epoch 2/100
1375/1375 [==============================] - 129s 94ms/step - loss: 1.6214 - accuracy: 0.3841 - val_loss: 1.5882 - val_accuracy: 0.3547
Epoch 3/100
1375/1375 [==============================] - 129s 94ms/step - loss: 1.5801 - accuracy: 0.4117 - val_loss: 1.5167 - val_accuracy: 0.3759
Epoch 4/100
1375/1375 [==============================] - 129s 94ms/step - loss: 1.5592 - accuracy: 0.4305 - val_loss: 1.2984 - val_accuracy: 0.5223
Epoch 5/100
1375/1375 [==============================] - 129s 94ms/step - loss: 1.5402 - accuracy: 0.4325 - val_loss: 1.4136 - val_accuracy: 0.4647
Epoch 6/100
1375/1375 [==============================] - 129s 94ms/step - loss: 1.5264 - accuracy: 0.4395 - val_loss: 1.3752 - val_accuracy: 0.4716
Epoch 7/100
1375/1375 [==============================] - 130s 94ms/step - loss: 1.5125 - accuracy: 0.4327 - val_

KeyboardInterrupt: 

In [331]:
del model